In [ ]:
!pip install sty

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import ast
from collections import namedtuple
from sty import bg, rs
from matplotlib import pyplot as plt

# **Features**

In [ ]:
features = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/features.csv")
features.head(5)

**Num features per case**

In [ ]:
cases = []
num_features = []
for x in set(features['case_num']):
    cases.append(x)
    num_features.append(features['case_num'].tolist().count(x))
plt.bar(cases, num_features)
plt.ylabel("num features")
plt.xlabel("case")
plt.show()

# **Patient notes**

In [ ]:
patient_notes = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv")
patient_notes.head(5)

**Patient notes per case**

In [ ]:
cases = []
num_notes = []
for x in set(patient_notes['case_num']):
    cases.append(x)
    num_notes.append(patient_notes['case_num'].tolist().count(x))
plt.bar(cases, num_notes)
plt.ylabel("patient notes")
plt.xlabel("case")
plt.show()

**Patient note lengths**

In [ ]:
for note in patient_notes['pn_history'][:1]:
    print(type(note))
note_lengths = [len(note) for note in patient_notes['pn_history']]
plt.hist(note_lengths)
plt.ylabel("Patient Note Length")
plt.yscale('log')
plt.show()

# **Train set**

In [ ]:
train = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/train.csv")
train['annotation'] = train['annotation'].apply(ast.literal_eval)
train['location'] = train['location'].apply(ast.literal_eval)
train.head(5)

**Annotation lengths**

In [ ]:
annotation_lengths = [len(x) for an in train['annotation'].tolist() for x in an]
plt.hist(annotation_lengths)
plt.xlabel("Annotation Length")
plt.yscale('log')
plt.show()

**Num annotated notes per case**

In [ ]:
cases = list(set(train['case_num']))
patient_notes_num = [0 for x in cases]
for x in set(train['pn_num']):
    case_num = train[train['pn_num'] == x]['case_num'].tolist()[0]
    patient_notes_num[case_num]+=1
plt.bar(cases, patient_notes_num)
plt.xlabel("Case")
plt.ylabel("Annotated notes")
plt.show()

# **Note with highlighted features**

In [ ]:
CaseData = namedtuple('CaseData', 'feature location num')
def getData(patient_note_num, case_num):
    patient_note=""
    for index, row in patient_notes.iterrows():
        if row['pn_num'] == patient_note_num and row['case_num'] == case_num:
            patient_note = row['pn_history']
    case_datas = []
    for index, row in train.iterrows():
        if row['pn_num'] == patient_note_num and row['case_num'] == case_num:
            ft_num = row['feature_num']
            for i in range(len(row['annotation'])):
                case_datas.append(CaseData(row['annotation'][i], list(map(int, row['location'][i].split(" "))), ft_num))
    return patient_note, case_datas
    
    

In [ ]:
patient_note, case_datas = getData(patient_note_num = 16, case_num = 0)
print(len(case_datas))
last_idx=0
for data in sorted(case_datas, key = lambda case_data : case_data.location[0]):
    print(patient_note[last_idx : data.location[0]], end="")
    print(bg.blue + patient_note[data.location[0] : data.location[1]] + bg.rs, end="")
    last_idx = data.location[1]
    